In [3]:
import pandas as pd
from sqlalchemy import create_engine
import json

In [5]:
dfTitleBasics = pd.read_csv(r"C:\Users\tenai\Desktop\projets\projet_netfloux\projet_floux\Netfloox-Red-Generals\downloads\title.basics.tsv", encoding='utf8', nrows=1000000, delimiter='\t', na_values='\\N')
dfNameBasics = pd.read_csv(r"C:\Users\tenai\Desktop\projets\projet_netfloux\projet_floux\Netfloox-Red-Generals\downloads\name.basics.tsv", encoding='utf8', nrows=1000000, delimiter='\t', na_values='\\N')
dfTitlePrincipals = pd.read_csv(r"C:\Users\tenai\Desktop\projets\projet_netfloux\projet_floux\Netfloox-Red-Generals\downloads\title.principals.tsv", encoding='utf8', nrows=1000000, delimiter='\t', na_values='\\N')
dfRatings = pd.read_csv(r"C:\Users\tenai\Desktop\projets\projet_netfloux\projet_floux\Netfloox-Red-Generals\downloads\title.ratings.tsv", encoding='utf8', nrows=1000000, delimiter='\t', na_values='\\N')
dfAkas = pd.read_csv(r"C:\Users\tenai\Desktop\projets\projet_netfloux\projet_floux\Netfloox-Red-Generals\downloads\title.akas.tsv", encoding='utf8', nrows=1000000, delimiter='\t', na_values='\\N')

In [7]:
mergedTitleBasicsRatings = pd.merge(dfTitleBasics, dfRatings, how='inner', on='tconst')
#mergedTitlesAkas = pd.merge(mergedTitleBasicsRatings, dfAkas, how='inner', on='titleId')
mergedTitlePrincipals = pd.merge(mergedTitleBasicsRatings, dfTitlePrincipals, how='inner', on='tconst')
mergedDF = pd.merge(mergedTitlePrincipals, dfNameBasics, how='inner', on='nconst')

In [9]:
mergedDF = mergedTitleBasicsRatings.dropna(inplace=True)

In [6]:
with open('.env', 'r') as json_file:
    env = json.load(json_file)

db = create_engine(
    env["url"],
    connect_args={'sslmode':'require','options': '-csearch_path={}'.format(env["dbschema"])},
    echo=False,
).connect()

In [18]:
view = """
CREATE VIEW films_overview
SELECT *
FROM title_basics
LEFT JOIN title_principals ON "tconst" = title_principals."tconst"
"""

In [19]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor


In [20]:
scaler = StandardScaler()
encoder = OneHotEncoder()
vectorizer = CountVectorizer()
regressor = RandomForestRegressor(n_estimators= 100, max_depth=3, criterion="squared_error")
ct = ColumnTransformer([
    ('num', scaler, ['runtimeMinutes', 'startYear'])
    #('cat', encoder, ['genres'])
])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

pipe = Pipeline([
    ('transformer', ct),
    ('regressor', regressor)
    ])
pipe.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['runtimeMinutes',
                                                   'startYear'])])),
                ('regressor', RandomForestRegressor(max_depth=3))])

In [22]:
predictions = pipe.predict(X_test)

In [23]:
from sklearn.metrics import mean_absolute_error

In [24]:
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 0.8542244252044231


In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
param_grid = {'regressor__max_depth': [None, 10, 20, 30],
              'regressor__min_samples_split': [2, 5, 10],
              'regressor__min_samples_leaf': [1, 2, 4]}

In [27]:
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['runtimeMinutes',
                                                                          'startYear'])])),
                                       ('regressor',
                                        RandomForestRegressor(max_depth=3))]),
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__min_samples_leaf': [1, 2, 4],
                         'regressor__min_samples_split': [2, 5, 10]},
             scoring='neg_mean_absolute_error')

In [28]:
print("Meilleurs hyperparamètres :", grid_search.best_params_)

Meilleurs hyperparamètres : {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 5}


In [29]:
param_grid = {'regressor__max_depth': [10],
              'regressor__min_samples_split': [10],
              'regressor__min_samples_leaf': [4]}

In [30]:
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

In [31]:
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE) avec les meilleurs hyperparamètres : {mae}")

Mean Absolute Error (MAE) avec les meilleurs hyperparamètres : 0.8477520272022939
